In [18]:
import random
import numpy as np
from copy import deepcopy
import VNS_ILS_for_GA
import pandas as pd

In [19]:
def one_crossover(str1, n1, str2, n2):
    n = min(n1,n2)
    i = random.randint(1, n)
    return str1[:i]+str2[i:], str2[:i] + str1[i:]

In [20]:
def two_crossover(str1, n1, str2, n2):
    n = min(n1,n2)
    i = random.randint(0, n-1)
    j = random.randint(i+1, n)
    return str1[:i] + str2[i:j] + str1[j:], str2[:i] + str1[i:j] + str2[j:]

In [21]:
def uniform_crossover(str1, n1, str2, n2):
    n = min(n1,n2)
    s = ''
    for i in range(n):
        sum = int(str1[i]) + int(str2[i])
        if sum == 2:
            s+='1'
        elif sum == 0:
            s+='0'
        else:
            s+= str(random.randint(0, 1))
    if n1>n2:
        return s + str1[n:]
    else:
        return s + str2[n:]

In [22]:
def three_crossover(str1,n1,str2,n2,str3,n3):
    n = min(n1,n2,n3)
    s=''
    for i in range(n):
        sum = int(str1[i]) + int(str2[i]) + int(str3[i])
        if sum >= 2:
            s +='1'
        else:
            s +='0'
    return s

In [23]:
def cross(str1,n1,str2,n2):
    i = random.randint(1, n1)
    j = random.randint(1, n2)
    return str1[:i]+str2[j:], str2[:j] + str1[i:]

In [24]:
def point_mutation(s, n):
    i = random.randint(0, n - 1)
    return s[:i] + str((int(s[i])+1) % 2) + s[i+1:]

In [25]:
def block_mutation(s, n, proportion): # proportion - какая минимальная часть решения будет перемешана 
    size = round(n * proportion - 0.5) #минимальный размер перемешиваемого участка
    i = random.randint(0, n-size)
    j = random.randint(i+size, n)
    rstr = list(s[i:j])
    random.shuffle(rstr)
    return s[:i] + ''.join(rstr) + s[j:]

In [26]:
class Solution(object):
    def __init__(self, s):
        self.size = 13
        self.params = s[0:self.size]
        res = self.object_function()
        self.fitness = res[0]
        self.solution = res[1]
        self.int_params = res[2]
    def __repr__(self):
        return f"\nSolution {self.params, self.int_params, self.fitness}"
    
    def object_function(self):
        elem_in_cl = int(self.params[0:3],2) + 1 #не больше 8 элементов в начальных кластерах 
        repeat = int(self.params[3:6],2) + 1 #не больше 8 запуска алгоритма
#         repeat = 5
        vnd = int(self.params[6],2) # сначала смена строки или столбца
        shaking = int(self.params[7],2) # сначала merge \\ split 
        n_shaking_merge = int(self.params[8:11],2) + 1  #[1,8]сколько раз повторить shaking
        n_shaking_split = int(self.params[11:13],2) + 1 # [1,4]сколько раз запустить split/merge для поиска лучшего результата
        
        self.solution, self.fitness = VNS_ILS_for_GA.main(elem_in_cl, repeat, vnd, shaking, n_shaking_merge, n_shaking_split )     
        return self.fitness, self.solution, [elem_in_cl,repeat,vnd,shaking, n_shaking_merge,n_shaking_split]
        
    
    def __eq__(self, s2):
        return self.params == s2.params
    def __ne__(self, s2):
        return self.params != s2.params
        

In [32]:
Solution('1010111')


Solution ('101011', 6, 0.16326530612244897, [
Customer ([3, 17], [3, 10], 0, 4) 
0  0
0  0, 
Customer ([8, 16], [8, 4], 2, 2) 
1  0
0  1, 
Customer ([7, 19], [1, 2], 1, 3) 
0  0
0  1, 
Customer ([0, 14], [7, 0], 2, 2) 
0  1
1  0, 
Customer ([2, 15], [12, 14], 1, 3) 
1  0
0  0, 
Customer ([5], [15], 0, 1) 
0, 
Customer ([1], [5], 0, 1) 
0, 
Customer ([10, 11, 12, 6, 18, 13], [11, 17, 16, 13, 9, 6], 17, 19) 
0  1  0  1  0  0
1  1  1  1  0  0
0  1  1  0  1  0
0  1  0  1  0  1
0  0  0  1  1  0
0  1  1  1  0  0, 
Customer ([4], [19], 0, 1) 
0, 
Customer ([9], [18], 1, 0) 
1])

In [27]:
class Population(object):
    def __init__(self, size = 30 , chrom_size = 13, mut_prob = 3/30):
        self.epoch = 0
        self.generation = []
        self.parents = []
        self.size = size
        self.chrom_size = chrom_size
        self.mut_prob = mut_prob #какая часть от поколения мутирует
        
    def __repr__(self):
        return f"Population {self.epoch, self.size, self.mut_prob, self.generation}"
    
    def initial_generation(self):
        for _ in range(self.size * 2):
            self.generation.append(Solution(str(random.choices([0,1],k=self.chrom_size))[1:-1].replace(', ', '')))
    
    def selection(self):
        self.parents = []
        self.epoch += 1
        summ_fitness = sum([s.fitness for s in self.generation])
        prob = [s.fitness / summ_fitness for s in self.generation]
        for _ in range(self.size+1):           
            self.parents.append(list(np.random.choice(self.generation, 2, p = prob, replace=True)))
        
    def reproduction(self):
        str_for_creating = []
        operations = {0:one_crossover, 1:two_crossover, 2:uniform_crossover, 3:cross}
        for pair in self.parents:
            i = random.randint(0,3)
            str_for_creating += operations[i](pair[0].params,pair[0].size, pair[1].params, pair[1].size)
            
        listed_parents = []
        for s in self.parents:
            listed_parents += s
        str_for_creating = sorted(set(str_for_creating) - set([s.params for s in listed_parents]))
        self.generation = [Solution(s[0:13]) for s in str_for_creating if len(s)>=13]
        self.generation += listed_parents
        
        
    def mutation(self):
        n = round(self.mut_prob * len(self.generation) + 0.5)
        str_for_creating = []
        point_mutated = list(np.random.choice(self.generation, n))
        block_mutated = list(np.random.choice(self.generation, n))
        str_for_creating += [(point_mutation(m.params, m.size)) for m in point_mutated]
        str_for_creating += [(block_mutation(m.params, m.size, 0.3)) for m in block_mutated] 
        str_for_creating = sorted(set(str_for_creating))
        mut_sol = [Solution(s) for s in str_for_creating]
        self.generation += mut_sol
        print("MUTATED", mut_sol, '\n')
        #получается, что особи и мутируют, и нет. Но в любом случае generation сократится до конкретных размеров, взяв только лучших. Так что это не сыграет роли
                    
    def live(self):
        BSOLS = []
        self.initial_generation()
        print("INITIAL", self.generation)
        best_sol = deepcopy(self.generation[0])
        for s in self.generation:
            if best_sol.fitness < s.fitness:
                best_sol = deepcopy(s)
        i = 0
        while (i<5):
            i+=1
            self.selection()
            self.parents[-1][1] = best_sol
            print([(pair[0].fitness,pair[1].fitness) for pair in self.parents])
            self.reproduction()
            print("BEFORE MUTATION", len(self.generation))
            self.mutation()
            print("AFTER MUTATION", len(self.generation))
            cur_sol = deepcopy(self.generation[0])
            for s in self.generation:
                if s.fitness > cur_sol.fitness:
                    cur_sol = deepcopy(s)
            print("ALL GENERATION", [s.fitness for s in self.generation])
            print("THE BEST", cur_sol)
            if best_sol.fitness >= cur_sol.fitness:
                print("NOT UPDATED", end='\n\n')
            else:
                print("UPDATED", end='\n\n')
                best_sol = deepcopy(cur_sol)
            BSOLS.append(best_sol)
        self.generation.sort(key = lambda x: x.fitness)
        return best_sol, self.generation[-self.size:], [s.params for s in self.generation[-self.size:]], BSOLS
    
    

In [28]:
def get_answer(ans, file_name = '20x20.txt',):
    VNS_ILS_for_GA.file_name = file_name
    best_sol = []
    best_sum = -1
    for a in ans:
        s = Solution(a)
        if s.fitness > best_sum:
            best_sol = deepcopy(s)
            best_sum = deepcopy(s.fitness)
        print(s)
    print("BEST ", best_sol)

In [29]:
def get_table(strings):
    int_params=[]
    res = []
    str_params = []
    VNS_ILS_for_GA.file_name = '20x20.txt'
    for a in strings:
        s = Solution(a)
        int_params.append(str(s.int_params))
        res.append(s.fitness)
        str_params.append(a)
    table = pd.DataFrame({'binary string':str_params, 'int parametrs': int_params, '20x20':res })
    for name in names[1:]:
        res = []
        VNS_ILS_for_GA.file_name = name
        for a in strings:
            s = Solution(a)
            res.append(s.fitness)
        col = pd.DataFrame({name[:-4] : res})
        table = pd.concat([table, col], axis =1)
    aver = [np.mean(r[2:]) for r in table.values]
    return pd.concat([table, pd.DataFrame({"average":aver})], axis =1) 

In [30]:
STRINGS=[]
BSOLS_UPD = []
names =['20x20.txt','24x40.txt','30x50.txt','30x90.txt','37x53.txt']

In [31]:
for name in names:
    VNS_ILS_for_GA.file_name = name
    pop = Population(5)
    LIVE = pop.live()
    STRINGS+=(LIVE[2])
    BSOLS_UPD.append(LIVE[3])

INITIAL [
Solution ('1100101110001', [7, 3, 1, 1, 5, 2], 0.4236111111111111), 
Solution ('0010011111011', [2, 2, 1, 1, 7, 4], 0.4305555555555556), 
Solution ('1011111110111', [6, 8, 1, 1, 6, 4], 0.41830065359477125), 
Solution ('0000110100110', [1, 4, 0, 1, 2, 3], 0.4108527131782946), 
Solution ('1110101001001', [8, 3, 1, 0, 3, 2], 0.4222222222222222), 
Solution ('1001101000110', [5, 7, 1, 0, 2, 3], 0.41830065359477125), 
Solution ('0011000100001', [2, 5, 0, 1, 1, 2], 0.4222222222222222), 
Solution ('0001111010110', [1, 8, 1, 0, 6, 3], 0.4206896551724138), 
Solution ('0001110001001', [1, 8, 0, 0, 3, 2], 0.4233576642335766), 
Solution ('1101001111011', [7, 5, 1, 1, 7, 4], 0.41139240506329117)]
[(0.41830065359477125, 0.41830065359477125), (0.41139240506329117, 0.41830065359477125), (0.41830065359477125, 0.4206896551724138), (0.4206896551724138, 0.41830065359477125), (0.4236111111111111, 0.4305555555555556), (0.4222222222222222, 0.4305555555555556)]
BEFORE MUTATION 20
MUTATED [
Solution (

MUTATED [
Solution ('0000101011110', [1, 3, 1, 0, 8, 3], 0.44871794871794873), 
Solution ('1000110001110', [5, 4, 0, 0, 4, 3], 0.44594594594594594), 
Solution ('1111010111101', [8, 6, 0, 1, 8, 2], 0.45695364238410596), 
Solution ('1111110100011', [8, 8, 0, 1, 1, 4], 0.4319526627218935)] 

AFTER MUTATION 21
ALL GENERATION [0.4429530201342282, 0.43243243243243246, 0.4473684210526316, 0.45454545454545453, 0.45751633986928103, 0.4473684210526316, 0.4444444444444444, 0.44654088050314467, 0.44155844155844154, 0.44654088050314467, 0.45751633986928103, 0.4473684210526316, 0.46153846153846156, 0.44654088050314467, 0.4533333333333333, 0.4375, 0.46153846153846156, 0.44871794871794873, 0.44594594594594594, 0.45695364238410596, 0.4319526627218935]
THE BEST 
Solution ('1000110001111', [5, 4, 0, 0, 4, 4], 0.46153846153846156)
NOT UPDATED

[(0.4473684210526316, 0.44594594594594594), (0.44155844155844154, 0.46153846153846156), (0.4473684210526316, 0.4429530201342282), (0.4444444444444444, 0.44444444444

BEFORE MUTATION 17
MUTATED [
Solution ('1010010010010', [6, 2, 0, 0, 5, 3], 0.46629213483146065), 
Solution ('1100010001010', [7, 2, 0, 0, 3, 3], 0.4636363636363636), 
Solution ('1101001001110', [7, 5, 1, 0, 4, 3], 0.4900990099009901), 
Solution ('1101111001110', [7, 8, 1, 0, 4, 3], 0.48717948717948717)] 

AFTER MUTATION 21
ALL GENERATION [0.4279475982532751, 0.4720812182741117, 0.4918032786885246, 0.5026455026455027, 0.49246231155778897, 0.5052083333333334, 0.49743589743589745, 0.4439252336448598, 0.48663101604278075, 0.4603960396039604, 0.4603960396039604, 0.4603960396039604, 0.48128342245989303, 0.49743589743589745, 0.47593582887700536, 0.47593582887700536, 0.5053191489361702, 0.46629213483146065, 0.4636363636363636, 0.4900990099009901, 0.48717948717948717]
THE BEST 
Solution ('1110111100100', [8, 4, 1, 1, 2, 1], 0.5053191489361702)
NOT UPDATED

INITIAL [
Solution ('0110111100111', [4, 4, 1, 1, 2, 4], 0.45251396648044695), 
Solution ('1111000100010', [8, 5, 0, 1, 1, 3], 0.4385026737

BEFORE MUTATION 19
MUTATED [
Solution ('0000011010111', [1, 2, 1, 0, 6, 4], 0.5876923076923077), 
Solution ('0000011100011', [1, 2, 1, 1, 1, 4], 0.581151832460733), 
Solution ('0010101001011', [2, 3, 1, 0, 3, 4], 0.5996258185219832), 
Solution ('1011111000111', [6, 8, 1, 0, 2, 4], 0.5976597659765976)] 

AFTER MUTATION 23
ALL GENERATION [0.5827996340347666, 0.5916542473919523, 0.595256312165264, 0.6020128087831657, 0.5916030534351145, 0.6009345794392523, 0.6020128087831657, 0.5957983193277311, 0.5916030534351145, 0.6064220183486239, 0.5920948616600791, 0.5996326905417815, 0.6059204440333025, 0.5940366972477065, 0.5940366972477065, 0.5940366972477065, 0.5795620437956205, 0.5940366972477065, 0.6064220183486239, 0.5876923076923077, 0.581151832460733, 0.5996258185219832, 0.5976597659765976]
THE BEST 
Solution ('1111010101001', [8, 6, 0, 1, 3, 2], 0.6064220183486239)
NOT UPDATED

[(0.5940366972477065, 0.581151832460733), (0.5795620437956205, 0.581151832460733), (0.6059204440333025, 0.6020128

In [32]:
TABLE = get_table(['0101110000000'] + sorted(set(STRINGS)))
TABLE

,binary string,int parametrs,20x20,24x40,30x50,30x90,37x53,average
0,0101110000000,"[3, 8, 0, 0, 1, 1]",0.401639,0.415663,0.492228,0.441417,0.599638,0.470117
1,0001111011111,"[1, 8, 1, 0, 8, 4]",0.422535,0.448276,0.502618,0.468927,0.594495,0.487370
2,0010011100011,"[2, 2, 1, 1, 1, 4]",0.408451,0.436620,0.494681,0.465969,0.591654,0.479475
3,0010011100111,"[2, 2, 1, 1, 2, 4]",0.389706,0.443662,0.483516,0.448753,0.594037,0.471935
4,0010011111011,"[2, 2, 1, 1, 7, 4]",0.414966,0.455128,0.507937,0.473973,0.595256,0.489452
5,1000010110011,"[5, 2, 0, 1, 5, 4]",0.401361,0.454545,0.478873,0.445040,0.586924,0.473349
6,1000110001111,"[5, 4, 0, 0, 4, 4]",0.405229,0.442953,0.478947,0.451872,0.605920,0.476984
7,1000111111001,"[5, 4, 1, 1, 7, 2]",0.419580,0.437500,0.482051,0.454308,0.596313,0.477951
8,1001101011010,"[5, 7, 1, 0, 7, 3]",0.416058,0.458599,0.486339,0.460000,0.597976,0.483794
9,1001111111101,"[5, 8, 1, 1, 8, 2]",0.427632,0.454545,0.505495,0.475000,0.605436,0.493621


In [33]:
print(BSOLS_UPD)

[[
Solution ('0010011111011', [2, 2, 1, 1, 7, 4], 0.4305555555555556), 
Solution ('0010011111011', [2, 2, 1, 1, 7, 4], 0.4305555555555556), 
Solution ('0010011111011', [2, 2, 1, 1, 7, 4], 0.4305555555555556), 
Solution ('0010011111011', [2, 2, 1, 1, 7, 4], 0.4305555555555556), 
Solution ('1011111110111', [6, 8, 1, 1, 6, 4], 0.43448275862068964)], [
Solution ('1000110001111', [5, 4, 0, 0, 4, 4], 0.46153846153846156), 
Solution ('1000110001111', [5, 4, 0, 0, 4, 4], 0.46153846153846156), 
Solution ('1000110001111', [5, 4, 0, 0, 4, 4], 0.46153846153846156), 
Solution ('1000110001111', [5, 4, 0, 0, 4, 4], 0.46153846153846156), 
Solution ('1000110001111', [5, 4, 0, 0, 4, 4], 0.46153846153846156)], [
Solution ('1110111101100', [8, 4, 1, 1, 4, 1], 0.5052083333333334), 
Solution ('1110111100100', [8, 4, 1, 1, 2, 1], 0.5053191489361702), 
Solution ('1110111100100', [8, 4, 1, 1, 2, 1], 0.5053191489361702), 
Solution ('1110111100100', [8, 4, 1, 1, 2, 1], 0.5053191489361702), 
Solution ('1110111100

In [34]:
pd.DataFrame(data=[[it.fitness for it in instance] for instance in BSOLS_UPD], index = [name[:-4] for name in names])

,0,1,2,3,4
20x20,0.430556,0.430556,0.430556,0.430556,0.434483
24x40,0.461538,0.461538,0.461538,0.461538,0.461538
30x50,0.505208,0.505319,0.505319,0.505319,0.505319
30x90,0.466851,0.474576,0.474576,0.474576,0.474576
37x53,0.606422,0.606422,0.606422,0.606422,0.606422
